In [1]:
import ROOT
import numpy as np
import pandas as pd
import math

Welcome to JupyROOT 6.16/00


In [18]:
_verbos = 1

In [19]:
def AsimovSignificance(h_s, h_b, sigma):
    n_bins = h_s.GetNbinsX()
    sigma_b = sigma
    #ar_AS = pd.DataFrame()
    ar_AS = []
    for n in range(n_bins):
        #print n 
        s = h_s.GetBinContent(n)
        b = h_b.GetBinContent(n)
        #print s
        #print b
        AS = ROOT.RooStats.AsimovSignificance(s, b, sigma_b)
        #print AS
        ar_AS_ = np.array([s, b, AS, sigma_b])
        ar_AS.append(ar_AS_)
    return ar_AS

In [20]:
def Sensitivity(df, name):
    if _verbos > 1:
        print(df)
    df['AsimovSignificance2%s' %(name)] = df['AsimovSignificance%s' % (name)]**2 
    sen = df[(df['AsimovSignificance2%s' %(name)] >0) & (df['AsimovSignificance2%s' %(name)] < float('inf'))].sum()
    sen = sen['AsimovSignificance2%s' %(name)]
    return math.sqrt(sen)

In [21]:
def calculate(filename, channel, input_sigma):
    f_Zee = ROOT.TFile("%s" % (filename))
    h_s_low = ROOT.TH1F("h_s_low", "h_s_low", 20, -1, 1)
    h_s_med = ROOT.TH1F("h_s_med", "h_s_med", 20, -1, 1)
    h_s_high = ROOT.TH1F("h_s_high", "h_s_high", 20, -1, 1)

    reg_nam = ['SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee', 'SR_med_Zmm', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn','SR_med_Wen']
    sam_nam = ['WH_lep', 'ZH_lep', 'ggZH_lep']
    #sam_nam = ['ZH_lep', 'ggZH_lep']
    ptv = ['PTV_0_75_hbb', 'PTV_150_250_0J_hbb', 'PTV_150_250_GE1J_hbb', 'PTV_75_150_hbb', 'PTV_GT250_hbb']
    #ptv = ['PTV_0_75_hbb',   'PTV_75_150_hbb']

    for reg in reg_nam:
        for sam in sam_nam:
            if "%s" %(channel) not in reg: break
            for pt in ptv:
                if "low" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_low.Add(h_)
                if "med" in reg:
                    break
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_med.Add(h_)
                if "high" in reg:
                    h_ = ROOT.TH1F()
                    h_.Clear()
                    h_ = f_Zee.Get("BDT_%s_%s_%s"%(reg, sam, pt))
                    h_s_high.Add(h_)
    
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm', 'SR_med_Zee', 'SR_med_Zmm', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn','SR_med_Wen']
    region_name = [ 'SR_low_Zee', 'SR_low_Zmm', 'SR_med_Znn_0j','SR_med_Znn_ge1j', 'SR_high_Zee', 'SR_high_Zmm', 'SR_high_Znn', 'SR_high_Wmn', 'SR_high_Wen', 'SR_med_Wmn','SR_med_Wen']

    sample_name = ['TT', 'VVHF', 'VVLF', 'Wj0b', 'Wj1b', 'Wj2b', 'Zj0b', 'Zj1b', 'Zj2b', 's_Top']
    dict_AS = {}

    sigma_b = input_sigma
    for region in region_name:
        for sample in sample_name:
            if "%s" %(channel) not in region: break
            #print("BDT_%s_TT" % (region))
            #print("BDT_%s_%s" % (region, sample))
            if "low" in region:
                h_s = h_s_low.Clone()
            if "med" in region:
                h_s = h_s_med.Clone()
            if "high" in region:
                h_s = h_s_high.Clone()
            
            h_b = f_Zee.Get("BDT_%s_%s" % (region, sample))
            l = AsimovSignificance(h_s, h_b, sigma_b)
            df = pd.DataFrame(l, columns=['signal', 'background', 'AsimovSignificance%s' %(region+sample), 'sigma_b'])
            sen = Sensitivity(df, region+sample)
            dict_AS.update({region+'_'+sample : [sen, sigma_b]})

    df_AS = pd.DataFrame.from_dict(dict_AS, orient='index', columns=['AsimovSignificance', 'sigma_b'])

    return df_AS

In [22]:
test_Zee = calculate("vhbb_Zee-2017.root", "Zee", 10)

In [23]:
print(test_Zee)

                   AsimovSignificance  sigma_b
SR_low_Zee_Wj2b              0.032858       10
SR_high_Zee_Zj0b             0.131087       10
SR_high_Zee_VVHF             0.146171       10
SR_low_Zee_VVHF              0.899460       10
SR_low_Zee_VVLF              0.640657       10
SR_high_Zee_TT               0.090553       10
SR_low_Zee_Zj0b              0.784782       10
SR_low_Zee_Wj0b              0.008117       10
SR_low_Zee_Zj2b              0.658433       10
SR_high_Zee_s_Top            0.038826       10
SR_high_Zee_Zj1b             0.153579       10
SR_high_Zee_Wj0b             0.004525       10
SR_high_Zee_VVLF             0.086880       10
SR_high_Zee_Wj1b             0.000000       10
SR_high_Zee_Wj2b             0.000000       10
SR_low_Zee_TT                0.730971       10
SR_low_Zee_Wj1b              0.000000       10
SR_low_Zee_Zj1b              0.784248       10
SR_high_Zee_Zj2b             0.164508       10
SR_low_Zee_s_Top             0.769871       10


In [41]:
test_Zmm_0 = calculate("vhbb_Zmm-2017.root", "Zmm", 0)
test_Zmm_10 = calculate("vhbb_Zmm-2017.root", "Zmm", 10)

In [42]:
test_Zmm_0 = test_Zmm_0.drop(columns=['sigma_b'])
test_Zmm_0['AsimovSignificance_10'] = test_Zmm_10['AsimovSignificance']
test_Zmm_0['ratio'] = test_Zmm_0['AsimovSignificance_10']/test_Zmm_0['AsimovSignificance']
print(test_Zmm_0)


                   AsimovSignificance  AsimovSignificance_10     ratio
SR_high_Zmm_VVLF             3.173804               0.137642  0.043368
SR_high_Zmm_Zj0b             2.625846               0.184764  0.070364
SR_high_Zmm_Wj2b             0.000000               0.000000       NaN
SR_low_Zmm_TT                1.781547               0.986879  0.553945
SR_high_Zmm_s_Top            0.724859               0.048559  0.066991
SR_low_Zmm_Zj2b              1.204040               0.878503  0.729629
SR_high_Zmm_Zj1b             1.986834               0.201395  0.101365
SR_high_Zmm_Zj2b             1.026123               0.225710  0.219964
SR_low_Zmm_Wj1b              0.000000               0.000000       NaN
SR_low_Zmm_Wj2b              1.595338               0.047209  0.029592
SR_high_Zmm_Wj1b             0.000000               0.000000       NaN
SR_high_Zmm_VVHF             1.589894               0.214473  0.134898
SR_low_Zmm_VVHF              2.974906               1.270856  0.427192
SR_low